In [ ]:
import pandas as pd
import unicodedata
import re
import nltk

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import nltk

# Loading data via google drive
drive.mount('/content/drive')

In [ ]:
data_df = pd.read_csv(r"/content/drive/My Drive/data/sinhala-hate-speech-dataset.csv")
data_df = pd.DataFrame(data_df)
data_df

In [ ]:
def lineBreakReplace(text):
    return text.replace("\r\n", " ")

In [ ]:
data_df['comment'] =data_df.apply(lambda x: lineBreakReplace(x.comment), axis=1)

In [ ]:
def removeEscapSequences(text):
    return "".join(ch for ch in text if unicodedata.category(ch)[0]!="C")

In [ ]:
data_df['comment'] =data_df.apply(lambda x: removeEscapSequences(x.comment), axis=1)

In [ ]:
def remove_digits(text):
    return ''.join([i for i in text if not i.isdigit()])

In [ ]:
data_df['comment'] =data_df.apply(lambda x: removeEscapSequences(x.comment), axis=1)

In [ ]:
def removeHtmlTags(text):
    p = re.compile(r'<.*?>')
    return p.sub('', text)

In [ ]:

data_df['comment'] = data_df.apply(lambda x: removeHtmlTags(x.comment), axis=1)

In [ ]:
data_df.head()

In [ ]:
def removeSpecialCharacters(sentence,keep_apostrophes=True):
    sentence = sentence.strip()
    if keep_apostrophes:
        PATTERN = r'[?|$|&|*|%|@|(|)|~]'
        filtered_sentence = re.sub(PATTERN, r'', sentence)
        return filtered_sentence
    return sentence

In [ ]:
def replace_full_stops(sentence):
    return sentence.replace("."," ")

In [ ]:
data_df['comment'] = data_df.apply(lambda x: removeSpecialCharacters(x.comment), axis=1)
data_df['comment'] = data_df.apply(lambda x: replace_full_stops(x.comment), axis=1)

In [ ]:
def remove_english_letters(sentence):
    non_english_sentence = re.sub(r'[a-zA-Z]', '', sentence)
    return non_english_sentence

In [ ]:
data_df['comment']  = data_df['comment'] .apply(lambda x: remove_english_letters(x))

In [ ]:
lines = list(open('/content/drive/My Drive/data/stop-words-sinhala.txt',encoding="utf8"))
sinhala_stop_words = [re.sub("\n","",x) for x in lines]

In [ ]:
def tokenize_text(text):
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    return tokens

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
data_df['token_list']  = data_df['comment'] .apply(lambda x: tokenize_text(x))

In [ ]:
def remove_sinhala_stop_words(tokens):
    filtered_tokens = [token for token in tokens if token not in sinhala_stop_words]
    return filtered_tokens

In [ ]:

data_df['token_list_after_sw'] = data_df.apply(lambda x: remove_sinhala_stop_words(x.token_list), axis=1)

In [ ]:
def remove_two_letter_words(tokenized_words):
    # Use list comprehension to filter out words with length 2
    filtered_words = [word for word in tokenized_words if len(word) > 3]
    return filtered_words

In [ ]:
data_df['token_list_after_sw'] = data_df.apply(lambda x: remove_two_letter_words(x.token_list_after_sw), axis=1)

In [ ]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
data_df['pre_processed_text'] = data_df.apply(lambda x: TreebankWordDetokenizer().detokenize(x.token_list_after_sw), axis=1)

In [ ]:
list_of_words = ' '.join(data_df['pre_processed_text']).split()

In [ ]:
# from collections import Counter
# import numpy as np
# most_common_words = [word for word, count in Counter(list_of_words).items() if count in np.arange(50,100)]
# unique_words = [word for word, count in Counter(list_of_words).items() if count in [1,2,3]]
# all_words_to_remove = most_common_words+unique_words;
# def remove_most_common_unique_words(tokens):
#     filtered_tokens = [token for token in tokens if token not in all_words_to_remove]
#     return filtered_tokens

In [ ]:
#data_df['token_list_after_common_words'] = data_df.apply(lambda x: remove_most_common_unique_words(x.token_list_after_sw), axis=1)

In [ ]:
data_df['pre_processed_text'] = data_df.apply(lambda x: TreebankWordDetokenizer().detokenize(x.token_list_after_sw), axis=1)

In [ ]:
# list_of_words = ' '.join(data_df['pre_processed_text']).split()
# Counter(list_of_words)

In [ ]:
data_df['word_count'] = data_df['pre_processed_text'].str.split().str.len()
#data_df.head(40)

In [ ]:
def total_unique_words(words):
    return nltk.FreqDist(words)

print("Total number of unique words",total_unique_words(list_of_words))

Total number of unique words <FreqDist with 16629 samples and 54176 outcomes>


In [ ]:
def total_words(words):
    return(len(words))
print("Total number of words",total_words(list_of_words))

Total number of words 54176


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

In [ ]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 30.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
def tokenize_function(examples):
    return tokenizer(examples["pre_processed_text"], padding='max_length', truncation=True, max_length=180)

In [ ]:
from datasets import Dataset
h_dataset = Dataset.from_pandas(data_df)

In [ ]:
h_dataset

Dataset({
    features: ['id', 'comment', 'label', 'token_list', 'token_list_after_sw', 'pre_processed_text', 'word_count'],
    num_rows: 6345
})

In [ ]:
tokenized_dataset = h_dataset.map(tokenize_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/6345 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'comment', 'label', 'token_list', 'token_list_after_sw', 'pre_processed_text', 'word_count', 'input_ids', 'attention_mask'],
    num_rows: 6345
})

In [ ]:
data_df

,id,comment,label,token_list,token_list_after_sw,pre_processed_text,word_count
0,1,`හුදෙකලා´ එකේ `ද´ ඇහෙනවද කාටහරි,0,"[`, හුදෙකලා´, එකේ, `, ද´, ඇහෙනවද, කාටහරි]","[හුදෙකලා´, ඇහෙනවද, කාටහරි]",හුදෙකලා´ ඇහෙනවද කාටහරි,3
1,2,18ක පෙනුමක් තියෙන්නේ,0,"[18ක, පෙනුමක්, තියෙන්නේ]","[පෙනුමක්, තියෙන්නේ]",පෙනුමක් තියෙන්නේ,2
2,3,2020 එනවා කිව්නාට මෙහෙම එයි කියලා හිතුවේ නෑ පට්ට,0,"[2020, එනවා, කිව්නාට, මෙහෙම, එයි, කියලා, හිතුව...","[2020, එනවා, කිව්නාට, මෙහෙම, කියලා, හිතුවේ, පට්ට]",2020 එනවා කිව්නාට මෙහෙම කියලා හිතුවේ පට්ට,7
3,4,25 කැල්ල හම්මෝ,0,"[25, කැල්ල, හම්මෝ]","[කැල්ල, හම්මෝ]",කැල්ල හම්මෝ,2
4,5,25 කෙල්ල අයියලගෙ කාලද කොහෙද ඉන්නෙ,1,"[25, කෙල්ල, අයියලගෙ, කාලද, කොහෙද, ඉන්නෙ]","[කෙල්ල, අයියලගෙ, කාලද, කොහෙද, ඉන්නෙ]",කෙල්ල අයියලගෙ කාලද කොහෙද ඉන්නෙ,5
...,...,...,...,...,...,...,...
6340,6341,ෆට්ට ඒත් ඉතින් ඔහේ ඔක්සිජන් ඔනී නැතී සෙට් එකක්...,0,"[ෆට්ට, ඒත්, ඉතින්, ඔහේ, ඔක්සිජන්, ඔනී, නැතී, ස...","[ෆට්ට, ඔක්සිජන්, නැතී, සෙට්, එකක්, ඉන්නවා]",ෆට්ට ඔක්සිජන් නැතී සෙට් එකක් ඉන්නවා,6
6341,6342,ෆයිනලි,0,[ෆයිනලි],[ෆයිනලි],ෆයිනලි,1
6342,6343,ෆුකෙන් හිනා මේවට,1,"[ෆුකෙන්, හිනා, මේවට]","[ෆුකෙන්, හිනා, මේවට]",ෆුකෙන් හිනා මේවට,3
6343,6344,"ෆෝන් එක චාර්ජ් කරගනින්,29 පට්ට",0,"[ෆෝන්, එක, චාර්ජ්, කරගනින්,29, පට්ට]","[ෆෝන්, චාර්ජ්, කරගනින්,29, පට්ට]","ෆෝන් චාර්ජ් කරගනින්,29 පට්ට",4


In [ ]:
label_names = ["Non-Hate-Speech", "Hate-Speech"]
id2label = {idx:label for idx, label in enumerate(label_names)}
id2label

{0: 'Non-Hate-Speech', 1: 'Hate-Speech'}

In [ ]:
label2id = {v:k for k,v in id2label.items()}
label2id

{'Non-Hate-Speech': 0, 'Hate-Speech': 1}

In [ ]:
from transformers import AutoModelForSequenceClassification
num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=num_labels, label2id=label2id, id2label=id2label)

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
dataset=tokenized_dataset.shuffle().train_test_split(test_size=0.3)

In [ ]:
dataset["train"]

Dataset({
    features: ['id', 'comment', 'label', 'token_list', 'token_list_after_sw', 'pre_processed_text', 'word_count', 'input_ids', 'attention_mask'],
    num_rows: 4441
})

In [ ]:
 model_checkpoint = "xlm-roberta-base"

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
   # print(predictions)
    predictions = np.argmax(predictions, axis=1)
    print(labels)
    print(predictions)
    print("=============================")
    return {"Accuracy": accuracy_score(labels, predictions) }

In [ ]:
from transformers import TrainingArguments

model_name = model_checkpoint.split("/")[-1]
batch_size = 10
num_train_epochs = 20
logging_steps = len(dataset["train"]) // (batch_size * num_train_epochs)

args = TrainingArguments(
    output_dir=f"{model_name}-hate-speech-cls",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    #logging_steps=logging_steps,
    push_to_hub=False,
    load_best_model_at_end= True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.486256,0.853466
2,0.384100,0.401140,0.859769
3,0.346000,0.444671,0.864496
4,0.302800,0.493508,0.861345
5,0.256600,0.624936,0.868697
6,0.228800,0.709367,0.870273
7,0.185400,0.607099,0.880252
8,0.152600,0.910276,0.851366
9,0.141900,0.889613,0.876050
10,0.141900,0.941116,0.878151


[1 1 1 ... 1 0 0]
[1 1 1 ... 1 0 1]
[1 1 1 ... 1 0 0]
[1 1 1 ... 1 0 1]
[1 1 1 ... 1 0 0]
[1 1 1 ... 1 0 1]
[1 1 1 ... 1 0 0]
[1 1 1 ... 1 0 1]
[1 1 1 ... 1 0 0]
[1 1 1 ... 1 0 1]
[1 1 1 ... 1 0 0]
[1 1 1 ... 1 0 1]
[1 1 1 ... 1 0 0]
[1 1 1 ... 1 0 1]
[1 1 1 ... 1 0 0]
[1 1 0 ... 1 0 0]
[1 1 1 ... 1 0 0]
[1 1 1 ... 1 0 1]
[1 1 1 ... 1 0 0]
[1 1 1 ... 1 0 1]
[1 1 1 ... 1 0 0]
[1 1 1 ... 1 0 1]
[1 1 1 ... 1 0 0]
[1 1 1 ... 1 0 1]
[1 1 1 ... 1 0 0]
[1 1 1 ... 1 0 0]
[1 1 1 ... 1 0 0]
[1 1 1 ... 1 0 1]
[1 1 1 ... 1 0 0]
[1 1 1 ... 1 0 1]
[1 1 1 ... 1 0 0]
[1 1 1 ... 1 0 1]
[1 1 1 ... 1 0 0]
[1 1 1 ... 1 0 1]
[1 1 1 ... 1 0 0]
[1 1 1 ... 1 0 1]
[1 1 1 ... 1 0 0]
[1 1 1 ... 1 0 1]
[1 1 1 ... 1 0 0]
[1 1 1 ... 1 0 1]


TrainOutput(global_step=8900, training_loss=0.1437798152880722, metrics={'train_runtime': 1563.9104, 'train_samples_per_second': 56.794, 'train_steps_per_second': 5.691, 'total_flos': 8215848259128000.0, 'train_loss': 0.1437798152880722, 'epoch': 20.0})

In [ ]:
##push model to huggingface
#from huggingface_hub import login
#login()

In [ ]:
#model.push_to_hub("malinda135/xlm-roberta-base-sinhala-hate-speech-cls")


model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/malinda135/xlm-roberta-base-sinhala-hate-speech-cls/commit/6b181065748eaaca9eebc2ba915cfd85778948c9', commit_message='Upload XLMRobertaForSequenceClassification', commit_description='', oid='6b181065748eaaca9eebc2ba915cfd85778948c9', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
#tokenizer.push_to_hub("malinda135/xlm-roberta-base-sinhala-hate-speech-cls")

CommitInfo(commit_url='https://huggingface.co/malinda135/xlm-roberta-base-sinhala-hate-speech-cls/commit/c43140a0d00998a432b268af252c2455e8ae4122', commit_message='Upload tokenizer', commit_description='', oid='c43140a0d00998a432b268af252c2455e8ae4122', pr_url=None, pr_revision=None, pr_num=None)